# Spark RDD vs DataFrame: Performance & API Comparison

This notebook demonstrates the fundamental differences between RDDs and DataFrames using word count on the **Complete Works of Shakespeare**.

## Key Takeaways

| Aspect | RDD | DataFrame |
|--------|-----|------------|
| **Paradigm** | Imperative (HOW) | Declarative (WHAT) |
| **Optimization** | None - executes as written | Catalyst optimizer |
| **Execution** | Java object serialization | Tungsten + code generation |
| **Type Safety** | Compile-time (Scala) | Runtime (schema) |
| **Best For** | Low-level control, unstructured data | Structured/semi-structured data |

## Setup

In [ ]:
import time
import urllib.request
import itertools
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    explode, split, lower, col, regexp_replace, trim, length
)

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

In [ ]:
# Download Shakespeare text
FILEPATH = "shakespeare.txt"
URL = "https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt"

if not Path(FILEPATH).exists():
    print(f"Downloading from {URL}...")
    urllib.request.urlretrieve(URL, FILEPATH)

file_size = Path(FILEPATH).stat().st_size
print(f"File size: {file_size / 1024:.1f} KB ({file_size / 1024 / 1024:.2f} MB)")

In [ ]:
# Create Spark session
spark = SparkSession.builder \
    .appName("RDD_vs_DataFrame_Demo") \
    .master("local[*]") \
    .config("spark.driver.memory", "2g") \
    .config("spark.sql.shuffle.partitions", "8") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("WARN")

print(f"Spark version: {spark.version}")
print(f"Running on: {sc.master}")

---
## RDD Implementation

With RDDs, you specify **HOW** to process data step by step. Spark executes exactly what you write with no optimization.

In [ ]:
def word_count_rdd(filepath: str):
    """
    Classic RDD word count - imperative style.
    
    Step by step:
    1. textFile()     -> Read lines from file
    2. flatMap()      -> Split lines into words (one row per word)
    3. map()          -> Normalize each word
    4. filter()       -> Remove empty strings
    5. map()          -> Create (word, 1) pairs
    6. reduceByKey()  -> Sum counts per word
    7. sortBy()       -> Order by count
    """
    return (
        sc.textFile(filepath)
        .flatMap(lambda line: line.split())
        .map(lambda word: ''.join(c for c in word.lower() if c.isalpha()))
        .filter(lambda word: len(word) > 0)
        .map(lambda word: (word, 1))
        .reduceByKey(lambda a, b: a + b)
        .sortBy(lambda x: x[1], ascending=False)
    )

In [ ]:
# Execute and time
start = time.perf_counter()
rdd_result = word_count_rdd(FILEPATH).collect()
rdd_time = time.perf_counter() - start

print(f"RDD execution time: {rdd_time:.3f}s")
print(f"\nTop 10 words:")
for word, count in rdd_result[:10]:
    print(f"  {word:15} {count:,}")

---
## DataFrame Implementation

With DataFrames, you specify **WHAT** you want. The Catalyst optimizer decides **HOW** to execute efficiently.

In [ ]:
def word_count_dataframe(filepath: str):
    """
    DataFrame word count - declarative style.
    
    Benefits:
    - Catalyst optimizer rewrites query for efficiency
    - Tungsten engine: off-heap memory, code generation
    - Automatic predicate pushdown, column pruning
    """
    return (
        spark.read.text(filepath)
        # Split into words and explode to rows
        .select(explode(split(col("value"), r"\s+")).alias("word"))
        # Normalize: lowercase, keep only letters
        .select(regexp_replace(lower(col("word")), r"[^a-z]", "").alias("word"))
        # Filter empty
        .filter(length(col("word")) > 0)
        # Group and count
        .groupBy("word").count()
        # Sort
        .orderBy(col("count").desc())
    )

In [ ]:
# Execute and time
start = time.perf_counter()
df_result = word_count_dataframe(FILEPATH).collect()
df_time = time.perf_counter() - start

print(f"DataFrame execution time: {df_time:.3f}s")
print(f"\nTop 10 words:")
for row in df_result[:10]:
    print(f"  {row['word']:15} {row['count']:,}")

---
## 📦 Understanding Partitions

Partitions are the fundamental unit of parallelism in Spark. Each partition is processed by one task on one core. Understanding partitions helps you understand:

- **Why shuffles are expensive** — data must move between partitions across the network
- **How parallelism works** — more partitions = more parallel tasks (up to available cores)
- **What the execution plan means** — `hashpartitioning(word, 8)` means 8 partitions

In [ ]:
# Check partition counts at different stages
raw_rdd = sc.textFile(FILEPATH)
words_rdd = raw_rdd.flatMap(lambda line: line.split())
pairs_rdd = words_rdd.map(lambda w: (w.lower(), 1))
counts_rdd = pairs_rdd.reduceByKey(lambda a, b: a + b)

print("RDD Partition Counts at Each Stage:")
print(f"  After textFile():    {raw_rdd.getNumPartitions()} partitions")
print(f"  After flatMap():     {words_rdd.getNumPartitions()} partitions (narrow transform - no change)")
print(f"  After map():         {pairs_rdd.getNumPartitions()} partitions (narrow transform - no change)")
print(f"  After reduceByKey(): {counts_rdd.getNumPartitions()} partitions (shuffle may change this)")

In [ ]:
# DataFrame partition count
raw_df = spark.read.text(FILEPATH)
print(f"\nDataFrame Partitions: {raw_df.rdd.getNumPartitions()}")
print(f"Shuffle partitions (spark.sql.shuffle.partitions): {spark.conf.get('spark.sql.shuffle.partitions')}")

### Visualizing Data Distribution Across Partitions

Let's see how words are distributed across partitions **before** and **after** the shuffle. This illustrates why shuffles are necessary for aggregations.

In [ ]:
# Use glom() to see data grouped by partition
# glom() collects all elements in each partition into a list

# Sample: first 5 words from each partition BEFORE shuffle
pairs_sample = pairs_rdd.mapPartitionsWithIndex(
    lambda idx, it: [(idx, list(itertools.islice(it, 5)))]
).collect()

print("BEFORE SHUFFLE (reduceByKey): Words are scattered randomly")
print("=" * 60)
for partition_id, words in pairs_sample[:4]:  # Show first 4 partitions
    word_list = [w[0] for w in words]
    print(f"Partition {partition_id}: {word_list}")

print("\n→ Notice: the same word can appear in multiple partitions!")
print("  This is why we need a shuffle to group by key.")

In [ ]:
# Sample: first 5 words from each partition AFTER shuffle
counts_sample = counts_rdd.mapPartitionsWithIndex(
    lambda idx, it: [(idx, list(itertools.islice(it, 5)))]
).collect()

print("AFTER SHUFFLE (reduceByKey): Each word lives in exactly ONE partition")
print("=" * 60)
for partition_id, words in counts_sample[:4]:
    word_list = [f"{w[0]}({w[1]})" for w in words]
    print(f"Partition {partition_id}: {word_list}")

print("\n→ Now each word appears in only one partition with its total count.")
print("  The shuffle moved all occurrences of each word to the same partition.")

In [ ]:
# Visualize partition sizes
partition_sizes_before = pairs_rdd.glom().map(len).collect()
partition_sizes_after = counts_rdd.glom().map(len).collect()

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Before shuffle
ax1 = axes[0]
ax1.bar(range(len(partition_sizes_before)), partition_sizes_before, color='#e74c3c', edgecolor='black')
ax1.set_xlabel('Partition ID')
ax1.set_ylabel('Number of Records')
ax1.set_title('BEFORE Shuffle: (word, 1) pairs per partition', fontweight='bold')
ax1.set_xticks(range(len(partition_sizes_before)))

# After shuffle
ax2 = axes[1]
ax2.bar(range(len(partition_sizes_after)), partition_sizes_after, color='#3498db', edgecolor='black')
ax2.set_xlabel('Partition ID')
ax2.set_ylabel('Number of Records')
ax2.set_title('AFTER Shuffle: (word, count) pairs per partition', fontweight='bold')
ax2.set_xticks(range(len(partition_sizes_after)))

plt.tight_layout()
plt.savefig('partition_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nTotal records BEFORE shuffle: {sum(partition_sizes_before):,} (every word occurrence)")
print(f"Total records AFTER shuffle:  {sum(partition_sizes_after):,} (unique words only)")
print(f"\n→ The shuffle dramatically reduced data volume by pre-aggregating!")

### How Hash Partitioning Works

After `reduceByKey`, words are distributed using **hash partitioning**:

```
partition_id = hash(word) % num_partitions
```

This ensures:
1. **All occurrences of the same word go to the same partition** — required for correct aggregation
2. **Deterministic assignment** — same word always maps to same partition
3. **Roughly even distribution** — hash functions spread keys uniformly (ideally)

In [ ]:
# Demonstrate hash partitioning
num_partitions = counts_rdd.getNumPartitions()
sample_words = ['the', 'and', 'of', 'to', 'a', 'hamlet', 'romeo', 'juliet']

print(f"Hash partitioning with {num_partitions} partitions:")
print("=" * 40)
for word in sample_words:
    partition = hash(word) % num_partitions
    print(f"  '{word}' → partition {partition}")

print("\n→ This is why 'hashpartitioning(word, 8)' appears in the execution plan!")

### Narrow vs Wide Transformations

| Type | Examples | Shuffle? | Partition Change? |
|------|----------|----------|-------------------|
| **Narrow** | `map`, `filter`, `flatMap` | No | Data stays in same partition |
| **Wide** | `reduceByKey`, `groupByKey`, `join`, `sort` | Yes | Data moves between partitions |

**Rule of thumb**: If an operation needs to see data from multiple partitions to produce a result, it requires a shuffle.

---
## 🔍 The Key Insight: Query Execution Plans

This is where the real difference becomes visible. The DataFrame's `explain()` shows how Catalyst transforms your query through multiple stages of optimization.

In [ ]:
# Create query (lazy - not executed yet)
df_query = word_count_dataframe(FILEPATH)

# Show the full execution plan
print("=" * 70)
print("DATAFRAME EXECUTION PLAN")
print("=" * 70)
df_query.explain(mode="formatted")

### Reading the DataFrame Physical Execution Plan

This plan shows **exactly how Spark will execute your DataFrame query**. Read it **bottom-up** — data flows from step (1) to step (10).

---

#### Step-by-Step Breakdown

**`(1) Scan text`** — Reads the raw text file. Each line becomes one row with a single column called `value`. The `ReadSchema` confirms we're only reading what we need (column pruning).

**`(2) Generate`** — The `explode(split(...))` operation. Splits each line on whitespace and "explodes" the resulting array so each word becomes its own row. One line with 10 words → 10 rows.

**`(3) Filter`** — Removes empty strings. Notice the condition combines `lower()`, `regexp_replace()`, and `length()` — Catalyst **fused these operations** rather than doing three separate passes.

**`(4) Project`** — Applies the lowercase + regex cleanup transformation. This is the normalized word output.

**`(5) HashAggregate` (partial)** — This is the first half of a **two-phase aggregation**. Each partition computes local counts *before* shuffling. The `partial_count` means "count what I have locally." This is a major optimization — instead of shuffling every word, we shuffle pre-aggregated `(word, partial_count)` pairs.

**`(6) Exchange`** — ⚠️ **SHUFFLE**. Data is redistributed across the cluster using `hashpartitioning(word, 8)`. All occurrences of the same word end up on the same partition. Shuffles are expensive (network I/O), so Spark minimizes them.

**`(7) HashAggregate` (final)** — Combines the partial counts from all partitions into final counts per word.

**`(8) Exchange`** — Another shuffle, this time using `rangepartitioning` to prepare for global sorting. Data is distributed so partition 1 has the highest counts, partition 2 the next highest, etc.

**`(9) Sort`** — Sorts within each partition. Because of the range partitioning, the final result is globally sorted.

**`(10) AdaptiveSparkPlan`** — Wrapper indicating **Adaptive Query Execution (AQE)** is enabled. Spark can modify this plan at runtime based on actual data statistics (e.g., adjusting partition sizes after seeing shuffle data).

---

#### Key Concepts Illustrated

| Concept | Where You See It |
|---------|------------------|
| **Two-phase aggregation** | Steps 5 → 6 → 7 (partial agg, shuffle, final agg) |
| **Predicate pushdown / fusion** | Step 3 combines multiple operations |
| **Shuffle = Exchange** | Steps 6 and 8 — the expensive operations |
| **Adaptive Query Execution** | Step 10 wrapper |

In [ ]:
# RDDs have no equivalent - you can only see the DAG lineage
rdd_query = word_count_rdd(FILEPATH)

print("=" * 70)
print("RDD DEBUG STRING (Lineage only - no optimization info)")
print("=" * 70)
print(rdd_query.toDebugString().decode('utf-8'))

### Reading the RDD Lineage (Debug String)

This shows the **DAG (Directed Acyclic Graph)** of RDD transformations — essentially a record of *what you wrote*, not an optimized execution plan. Read it **bottom-up**, following the indentation.

---

#### Step-by-Step Breakdown

**`HadoopRDD` → `MapPartitionsRDD`** — `sc.textFile()` reads the file. HadoopRDD handles the raw file read, then it's immediately wrapped in a MapPartitionsRDD to produce string lines.

**`PythonRDD`** — Your `flatMap()`, `map()`, `filter()`, and second `map()` are all here. PySpark batches consecutive narrow transformations into a single Python worker call.

**`PairwiseRDD`** — Preparation for `reduceByKey()`. Converts the Python `(word, 1)` tuples into a format Spark can shuffle.

**`ShuffledRDD`** — ⚠️ **SHUFFLE #1** for `reduceByKey()`. All identical keys are gathered to the same partition.

**`MapPartitionsRDD` → `PythonRDD`** — Post-shuffle processing and setup for sorting.

**`PairwiseRDD`** — Preparation for `sortBy()`. Samples data to determine range boundaries.

**`ShuffledRDD`** — ⚠️ **SHUFFLE #2** for `sortBy()`. Range partitioning to achieve global sort order.

**`MapPartitionsRDD` → `PythonRDD`** — Final sort within each partition and conversion back to Python objects.

---

#### What's Missing (Compared to DataFrame)

| Aspect | DataFrame Plan | RDD Lineage |
|--------|----------------|-------------|
| **Optimization details** | Shows predicate fusion, two-phase aggregation | Just shows "PythonRDD" — no insight into what's inside |
| **Column pruning** | Explicit `ReadSchema` | N/A (no schema) |
| **Aggregation strategy** | Clearly shows partial → shuffle → final | Hidden inside opaque Python calls |
| **Operation fusion** | Visible in plan | Happens implicitly, not shown |
| **Adaptive execution** | AQE wrapper present | Not available |

---

#### The Key Insight

Notice how your Python transformations collapse into opaque `PythonRDD` blocks. Spark can't see inside them — it just knows "call this Python function." This means:

1. **No Catalyst optimization** — Spark executes exactly what you wrote
2. **No Tungsten benefits** — Data serializes between JVM and Python workers
3. **No introspection** — You can't see if your code is efficient

The `(2)` at the start of each line indicates **2 partitions**. That's about all the runtime insight you get.

---

#### When This Matters

For simple jobs, the difference is minor. But imagine you wrote an inefficient filter after a join — DataFrame's Catalyst would push the filter before the join automatically. With RDDs, you'd get exactly the inefficient plan you wrote, and this debug string wouldn't even show you that it's a problem.

---
## Benchmarking: Multiple Runs

In [ ]:
def benchmark(name: str, func, n_runs: int = 5, warmup: int = 1):
    """Run benchmark with warmup."""
    # Warmup
    for _ in range(warmup):
        func()
    
    # Timed runs
    times = []
    for i in range(n_runs):
        start = time.perf_counter()
        result = func()
        times.append(time.perf_counter() - start)
    
    return {
        "name": name,
        "times": times,
        "mean": np.mean(times),
        "std": np.std(times),
        "min": np.min(times),
        "max": np.max(times)
    }

In [ ]:
N_RUNS = 5

print(f"Running {N_RUNS} benchmark iterations (plus warmup)...\n")

rdd_bench = benchmark(
    "RDD", 
    lambda: word_count_rdd(FILEPATH).collect(),
    n_runs=N_RUNS
)
print(f"RDD:       {rdd_bench['mean']:.3f}s ± {rdd_bench['std']:.3f}s")

df_bench = benchmark(
    "DataFrame",
    lambda: word_count_dataframe(FILEPATH).collect(),
    n_runs=N_RUNS
)
print(f"DataFrame: {df_bench['mean']:.3f}s ± {df_bench['std']:.3f}s")

speedup = rdd_bench['mean'] / df_bench['mean']
print(f"\n🚀 DataFrame is {speedup:.2f}x faster")

---
## Visualization

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# --- Performance Comparison ---
ax1 = axes[0]
names = ["RDD", "DataFrame"]
means = [rdd_bench["mean"], df_bench["mean"]]
stds = [rdd_bench["std"], df_bench["std"]]
colors = ["#e74c3c", "#3498db"]

bars = ax1.bar(names, means, yerr=stds, capsize=8, color=colors, 
               edgecolor="black", linewidth=1.5)

ax1.set_ylabel("Execution Time (seconds)", fontsize=12)
ax1.set_title("Word Count Performance: RDD vs DataFrame", fontsize=14, fontweight="bold")

# Add value labels
for bar, mean, std in zip(bars, means, stds):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + std + 0.05,
            f"{mean:.3f}s", ha="center", fontsize=12, fontweight="bold")

# Add speedup annotation
ax1.annotate(f"{speedup:.1f}x faster", 
            xy=(1, df_bench["mean"]), 
            xytext=(0.5, (rdd_bench["mean"] + df_bench["mean"])/2),
            fontsize=14, fontweight="bold", color="#27ae60",
            arrowprops=dict(arrowstyle="->", color="#27ae60", lw=2))

# --- Top Words ---
ax2 = axes[1]
top_words = df_result[:15]
words = [row["word"] for row in top_words]
counts = [row["count"] for row in top_words]

y_pos = np.arange(len(words))
ax2.barh(y_pos, counts, color="#3498db", edgecolor="black", linewidth=0.5)
ax2.set_yticks(y_pos)
ax2.set_yticklabels(words, fontsize=11)
ax2.invert_yaxis()
ax2.set_xlabel("Frequency", fontsize=12)
ax2.set_title("Top 15 Words in Shakespeare", fontsize=14, fontweight="bold")

for i, count in enumerate(counts):
    ax2.text(count + 200, i, f"{count:,}", va="center", fontsize=9)

plt.tight_layout()
plt.savefig("benchmark_results.png", dpi=150, bbox_inches="tight")
plt.show()

---
## Why DataFrames Are Faster

### 1. Catalyst Optimizer
- **Predicate pushdown**: Filters moved closer to data source
- **Column pruning**: Only reads columns actually needed
- **Constant folding**: Evaluates constant expressions at compile time
- **Join reordering**: Optimizes join order based on statistics

### 2. Tungsten Execution Engine
- **Off-heap memory**: Avoids JVM garbage collection overhead
- **Cache-aware computation**: Optimized for CPU cache hierarchy
- **Whole-stage code generation**: Compiles query stages to optimized bytecode

### 3. Efficient Data Representation
- RDDs: Java objects with full serialization overhead
- DataFrames: Binary row format, compact and efficient

---
## When to Use RDDs

RDDs still have valid use cases:

1. **Unstructured data** that doesn't fit a columnar schema
2. **Low-level transformations** not expressible in DataFrame API
3. **Fine-grained partitioning control** for specific performance tuning
4. **Legacy code** that predates DataFrame API
5. **Type safety in Scala** with compile-time checking

For most structured data processing: **prefer DataFrames**.

In [ ]:
# Cleanup
spark.stop()
print("Spark session stopped.")